# Segmenting and Clustering Neighborhoods in Toronto

In this notebook, I will first scrape data from a website to get information on the neighborhoods in Toronto and then convert it into a pandas dataframe.  Then I will obtain the latitude and longitude of the neighborhoods.  Finally, i will use the Foursquare API to explore the neighborhoods and use the data to segment and cluster the neighborhoods.

Import all the necessary libraries.

In [1]:
import numpy as np
import pandas as pd

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

!conda install -c anaconda beautifulsoup4
from bs4 import BeautifulSoup

print("Libraries imported!")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## Getting the data

I am going to use the BeautifulSoup library to scrape the website to get the data related to the Toronto neighborhoods.

In [12]:
# Get the source code
result=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(result,"lxml")

There is only one table, so it can be located easily.

In [84]:
table=soup.find("table")

In [113]:
#Convert it into a pandas dataframe
df=pd.read_html(str(table))[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [114]:
df.rename(columns={"Postcode":"PostalCode"},inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Clean the dataframe.

In [115]:
# Remove rows with borough 'Not assigned'
df=df[df["Borough"]!="Not assigned"].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [116]:
# Combine Neighborhoods with the same postalcode.
df=df.groupby("PostalCode").agg({"Borough":"first","Neighbourhood":",".join}).reset_index()
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [117]:
df

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


The neighborhood is 'Not assigned' for index 85.

In [118]:
# Change the name of neighborhoods that are 'Not assigned' to the name of the borough
df.loc[df["Neighbourhood"]=="Not assigned","Neighbourhood"]=df["Borough"]
df

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In index 85, the neighborhood name has been changed to the name of the corresponding borough.

In [119]:
#Print the no. of rows in the df
print(df.shape[0])

103


## Get the latitude and longitude for each postal code

I am going to use a csv file to get the latitude and longitude values.

In [120]:
#Get the latitude and longitude for each postal code
latlng_df=pd.read_csv("http://cocl.us/Geospatial_data")
latlng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [121]:
#Merge df and latlng_df into a single dataframe.
latlng_df.drop("Postal Code",axis=1,inplace=True)
df=pd.concat([df,latlng_df],axis=1)
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Explore neighborhoods in Toronto

To make the analysis simpler, I will only consider the boroughs with the word Toronto.

In [145]:
indices=[]
for i,bor in enumerate(df["Borough"]):
    if("Toronto" in bor):
        indices.append(i)
toronto_df=df.iloc[indices,:]
toronto_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [146]:
print(toronto_df.shape[0])

38


### Foursquare Credentials

In [147]:
# The code was removed by Watson Studio for sharing.

Function to get the category of nearby venues

In [153]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood PostalCode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a new dataframe called toronto_venues with the information of nearby venues

In [154]:
toronto_venues=getNearbyVenues(toronto_df["PostalCode"],toronto_df["Latitude"],toronto_df["Longitude"])

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y


In [155]:
print(toronto_venues.shape)
toronto_venues.head()

(842, 7)


,Neighborhood PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


The no. of venues returned for each neighborhood.

In [156]:
toronto_venues.groupby("Neighborhood PostalCode").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood PostalCode,,,,,,
M4E,5,5,5,5,5,5
M4K,30,30,30,30,30,30
M4L,22,22,22,22,22,22
M4M,30,30,30,30,30,30
M4N,3,3,3,3,3,3
M4P,9,9,9,9,9,9
M4R,22,22,22,22,22,22
M4S,30,30,30,30,30,30
M4T,2,2,2,2,2,2


No. of unique venue categories.

In [158]:
print("There are {} unique venue categories".format(len(toronto_venues["Venue Category"].unique())))

There are 187 unique venue categories


## Analyze the neighborhoods

In [159]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood PostalCode'] = toronto_venues['Neighborhood PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [160]:
toronto_onehot.shape

(842, 188)

Group the dataframe by the postal code and calculate the mean of the frequency of occurrence of each category

In [161]:
toronto_grouped=toronto_onehot.groupby("Neighborhood PostalCode").mean().reset_index()
toronto_grouped.head()

,Neighborhood PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.033333
2,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [162]:
toronto_grouped.shape

(38, 188)

Function to sort the venues in decreasing order of frequency

In [163]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

New dataframe with top 10 venues for each neighborhood.

In [208]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood PostalCode'] = toronto_grouped['Neighborhood PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Other Great Outdoors,Trail,Pub,Neighborhood,Yoga Studio,Creperie,Eastern European Restaurant,Dog Run,Discount Store
1,M4K,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Dessert Shop,Brewery,Bubble Tea Shop,Restaurant,Pub,Cosmetics Shop
2,M4L,Park,Sandwich Place,Liquor Store,Food & Drink Shop,Fish & Chips Shop,Italian Restaurant,Steakhouse,Intersection,Sushi Restaurant,Ice Cream Shop
3,M4M,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Cheese Shop,Ice Cream Shop,Sandwich Place,Latin American Restaurant,Bookstore
4,M4N,Bus Line,Park,Swim School,Yoga Studio,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


## Cluster the neighborhoods

Use k-means clustering to cluster the neighborhoods into 5 clusters.

In [209]:
# set number of clusters
k = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 3, 0, 0, 0, 2, 0], dtype=int32)

Create a new dataframe that contains the cluster and top 10 venues for each neighborhood.

In [210]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_df to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood PostalCode'), on='PostalCode')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Other Great Outdoors,Trail,Pub,Neighborhood,Yoga Studio,Creperie,Eastern European Restaurant,Dog Run,Discount Store
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Dessert Shop,Brewery,Bubble Tea Shop,Restaurant,Pub,Cosmetics Shop
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Liquor Store,Food & Drink Shop,Fish & Chips Shop,Italian Restaurant,Steakhouse,Intersection,Sushi Restaurant,Ice Cream Shop
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Cheese Shop,Ice Cream Shop,Sandwich Place,Latin American Restaurant,Bookstore
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Bus Line,Park,Swim School,Yoga Studio,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


## Visualize the results

Get latitude and longitude for Toronto using the geopy library.

In [211]:
address="Toronto, Canada"
geolocator=Nominatim(user_agent="toronto_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print("The geographical co-ordinates of Toronto are {}, {}".format(latitude,longitude))

The geographical co-ordinates of Toronto are 43.653963, -79.387207


Visualize the resulting clusters of the neighborhood using Folium.

In [212]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

### Cluster 1

In [213]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,0,Health Food Store,Other Great Outdoors,Trail,Pub,Neighborhood,Yoga Studio,Creperie,Eastern European Restaurant,Dog Run,Discount Store
41,M4K,East Toronto,"The Danforth West,Riverdale",0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Dessert Shop,Brewery,Bubble Tea Shop,Restaurant,Pub,Cosmetics Shop
42,M4L,East Toronto,"The Beaches West,India Bazaar",0,Park,Sandwich Place,Liquor Store,Food & Drink Shop,Fish & Chips Shop,Italian Restaurant,Steakhouse,Intersection,Sushi Restaurant,Ice Cream Shop
43,M4M,East Toronto,Studio District,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Cheese Shop,Ice Cream Shop,Sandwich Place,Latin American Restaurant,Bookstore
45,M4P,Central Toronto,Davisville North,0,Gym,Dance Studio,Breakfast Spot,Park,Clothing Store,Hotel,Food & Drink Shop,Restaurant,Sandwich Place,Eastern European Restaurant
46,M4R,Central Toronto,North Toronto West,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Burger Joint,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Park
47,M4S,Central Toronto,Davisville,0,Dessert Shop,Sandwich Place,Gym,Coffee Shop,Italian Restaurant,Pizza Place,Café,Sushi Restaurant,Pharmacy,Seafood Restaurant
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0,Coffee Shop,Pub,Pizza Place,Fried Chicken Joint,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Bagel Shop
51,M4X,Downtown Toronto,"Cabbagetown,St. James Town",0,Coffee Shop,Café,Restaurant,Italian Restaurant,Beer Store,Japanese Restaurant,Sandwich Place,Liquor Store,Diner,Butcher
52,M4Y,Downtown Toronto,Church and Wellesley,0,Gay Bar,Gastropub,Restaurant,Italian Restaurant,Japanese Restaurant,Bookstore,Breakfast Spot,Juice Bar,Bubble Tea Shop,Salon / Barbershop


### Cluster 2

In [214]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,M5N,Central Toronto,Roselawn,1,Pool,Garden,Yoga Studio,Cuban Restaurant,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner


### Cluster 3

In [215]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,M4T,Central Toronto,"Moore Park,Summerhill East",2,Tennis Court,Playground,College Gym,Colombian Restaurant,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner


### Cluster 4

In [216]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,M4N,Central Toronto,Lawrence Park,3,Bus Line,Park,Swim School,Yoga Studio,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


### Cluster 5

In [217]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,4,Park,Playground,Trail,Building,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner
64,M5P,Central Toronto,"Forest Hill North,Forest Hill West",4,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dance Studio,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store
